In [1]:
import pandas as pd
import numpy as np
import altair as alt
from dlq_risk_calcuation import LoanDlqRisks
from panel.interact import interact
import streamlit as st
import pickle

In [2]:
from sqlalchemy import URL, create_engine, text
alt.data_transformers.disable_max_rows()

hostname = 'database-3.cluster-cgldqbnitk50.us-east-1.rds.amazonaws.com'
username = 'postgres'
password = 'cas_analytics'
database = 'postgres'

url_object = URL.create(
    "postgresql",
    username=username,
    password=password,  # plain (unescaped) text
    host=hostname,
    database=database,
)

engine = create_engine(url_object)

# load the data
query = f"""
WITH loan_stats AS (
    SELECT 
        "Loan Identifier",
        "Original Interest Rate",
        "Original UPB",
        "Original Loan to Value Ratio (LTV)",
        "Original Combined Loan to Value Ratio (CLTV)",
        "Borrower Credit Score At Issuance",
        "Debt-To-Income (DTI)",
        "Zip Code Short",
        ROUND(
            1.0*SUM(CASE WHEN "Current Loan Delinquency Status" > 0 THEN 1 ELSE 0 END)/COUNT("Monthly Reporting Period"),
        2
        ) AS pct_dlq_marks
    FROM Cas.ref  
    GROUP BY 
        "Loan Identifier",
        "Original Interest Rate",
        "Original UPB",
        "Original Loan to Value Ratio (LTV)",
        "Original Combined Loan to Value Ratio (CLTV)",
        "Borrower Credit Score At Issuance",
        "Debt-To-Income (DTI)",
        "Zip Code Short"
)

SELECT * FROM loan_stats
"""

df = pd.DataFrame(engine.connect().execute(text(query)))

with open('../outputs/weights.pkl', 'rb') as file:
    weights = pickle.load(file)

In [5]:
interest_rate=st.number_input('What is the interest rate for this loan? Enter the percentage (e.g. 2% : 2.0)', value=3.5)
upb=st.number_input('What is the total loan amount in dollars?', value=20000)
ltv=st.number_input('What is the Loan-to-Value ratio? Enter percentage number (e.g. 45% -> 45)', value=80)
credit_score=st.number_input('What is the FICO score for the main borrower?',value=780)
dti=st.number_input('What is the Debt-To-Income ratio? Enter percentage (e.g. 20% -> 20)', value=22)
zipcode=st.number_input('What is the short Zipcode where the property is localted at?', value=244)

st.altair_chart(
    LoanDlqRisks(
        interest_rate,
        upb,
        ltv,
        credit_score,
        dti,
        zipcode
    ).get_relative_stats(ref_table=df)
    .generate_risk_summary(weight_params=weights)
    .diag_chart, 
    use_container_width=True
)

Too few samples for zipcode starting 244, using all data instead


DeltaGenerator()